# Гібридна постквантова криптографічна схема цифрового підпису: ДСТУ 4145 + ML-DSA-44

## 1. Імпорт та налаштування

In [2]:
import sys
from pathlib import Path
import time
import secrets
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, str(Path.cwd()))

# Імпортуємо модулі
from dstu4145.curve import get_dstu_curve_257
from hybrid import create_hybrid_scheme

print("Імпорт успішний!")

Імпорт успішний!


## 2. Генерація базової точки GF(2^257) ДСТУ 4145 (опційно: з перевіркою)

In [3]:
print("="*70)
print("ГЕНЕРАЦІЯ БАЗОВОЇ ТОЧКИ GF(2^257)")
print("="*70)

# Генерація базової точки з таймером
start_time = time.time()
field, curve, base_point, n = get_dstu_curve_257()
generation_time = time.time() - start_time

# Координати базової точки
xP = field.element_to_int(base_point.x)
yP = field.element_to_int(base_point.y)

print(f"\nЧас генерації базової точки: {generation_time:.4f} сек")
print(f"\nПараметри кривої GF(2^257):")
print(f"  Поле: GF(2^{field.m})")
print(f"  Порядок n: {hex(n)}")
print(f"\nКоординати базової точки P:")
print(f"  xP = {hex(xP)}")
print(f"  yP = {hex(yP)}")

# Перевірка
#on_curve = curve.is_on_curve(base_point)
#print(f"\nПеревірка: Точка на кривій = {on_curve}")

print("\n" + "="*70)

ГЕНЕРАЦІЯ БАЗОВОЇ ТОЧКИ GF(2^257)

Час генерації базової точки: 194.3111 сек

Параметри кривої GF(2^257):
  Поле: GF(2^257)
  Порядок n: 0x800000000000000000000000000000006759213af182e987d3e17714907d470d

Координати базової точки P:
  xP = 0x9ede5f8bc34a25ee763d2b00b9d8fb8fd417bf7c012fcdcad63bad439dae7f10
  yP = 0xef2978536b93bb0852614d61ebfb9c0cb5a1d8e25f94b51997c0bdcbb7efd2d4



## 3. Ініціалізація гібридної схеми

In [4]:
# Створюємо гібридну схему
scheme = create_hybrid_scheme()

print("Гібридна схема ініціалізована")

PURE HYBRID SIGNATURE
Ініціалізація...

1. ДСТУ 4145-2002:
    Поле: GF(2^257)
    Крива: y^2 + xy = x^3 + 0x^2 + B
    Порядок: 0x800000000000000000000000000000006759213af182e987d3e17714907d47...
    Хеш: Купина-256 (ДСТУ 7564:2014)

2. ML-DSA-44:
    ML-DSA ініціалізовано (використовується ML-DSA-44)
    Публічний ключ: 1312 байт
    Приватний ключ: 2560 байт
    Підпис: 2420 байт

 Гібридна схема готова (ДСТУ 4145(2^257) + ML-DSA-44 + Купина-256)
Гібридна схема ініціалізована


## 4. Генерація гібридної пари ключів

In [5]:
print("="*70)
print("ГЕНЕРАЦІЯ ГІБРИДНОЇ ПАРИ КЛЮЧІВ")
print("="*70)

# Генеруємо пару ключів з таймером
start_time = time.time()
keypair = scheme.generate_keypair(key_id="demo-257")
keygen_time = time.time() - start_time

print(f"\nЗагальний час генерації ключів: {keygen_time:.4f} сек")
print(f"\nІнформація про ключі:")
print(f"  ID: {keypair.key_id}")
print(f"  Створено: {keypair.created_at}")
print(f"\n  ДСТУ 4145 GF(2^257):")
print(f"    Приватний: {len(str(keypair.dstu_private_key))} цифр")
print(f"    Публічний: {len(keypair.dstu_public_key_bytes)} байт")
print(f"\n  ML-DSA-44:")
print(f"    Приватний: {len(keypair.mldsa_private_key)} байт")
print(f"    Публічний: {len(keypair.mldsa_public_key)} байт")

print("\n" + "="*70)

ГЕНЕРАЦІЯ ГІБРИДНОЇ ПАРИ КЛЮЧІВ

 Генерація гібридної пари ключів...
  1. ДСТУ 4145:
     Час генерації ключів ДСТУ 4145: 220.5889 сек
      Приватний ключ: 0x1372c05e4104ae99d3...
      Публічний ключ: 66 байт
  2. ML-DSA-44:
     Час генерації ключів ML-DSA-44: 0.0115 сек
      Приватний ключ: 2560 байт
      Публічний ключ: 1312 байт

 Гібридна пара ключів згенерована: demo-257

Загальний час генерації ключів: 220.6008 сек

Інформація про ключі:
  ID: demo-257
  Створено: 2026-02-22T13:42:49.104401

  ДСТУ 4145 GF(2^257):
    Приватний: 76 цифр
    Публічний: 66 байт

  ML-DSA-44:
    Приватний: 2560 байт
    Публічний: 1312 байт



## 5. Створення гібридного підпису

In [6]:
print("="*70)
print("СТВОРЕННЯ ГІБРИДНОГО ПІДПИСУ")
print("="*70)

# Дані для підпису
message = b"Simplicity is a difficult thing to achieve."

print(f"\nДані: {message.decode('utf-8')}")
print(f"Розмір: {len(message)} байт")

# Створюємо підпис (з таймером)
start_time = time.time()
signature = scheme.sign(message, keypair)
sign_time = time.time() - start_time

print(f"\nЗагальний час створення гібридного підпису: {sign_time:.4f} сек")
print(f"\nРозміри підписів:")
dstu_sig_size = len(scheme.dstu.export_signature(signature.dstu_signature))
print(f"  ДСТУ 4145: {dstu_sig_size} байт")
print(f"  ML-DSA-44: {len(signature.mldsa_signature)} байт")
print(f"  Гібридний: {dstu_sig_size + len(signature.mldsa_signature)} байт")

print("\n" + "="*70)

СТВОРЕННЯ ГІБРИДНОГО ПІДПИСУ

Дані: Simplicity is a difficult thing to achieve.
Розмір: 43 байт

  Створення гібридного підпису...
   Дані: 43 байт
   1. ДСТУ 4145...
     Час підпису за допомогою ДСТУ 4145: 230.6367 сек
       Підпис: (r=0x186a037e34d5eb297b..., s=0x186a57b9b02668a28c...)
   2. ML-DSA-44...
     Час підпису за допомогою ML-DSA-44: 0.0005 сек
       Підпис: 2420 байт

 Гібридний підпис створено (2484 байт)

Загальний час створення гібридного підпису: 230.6570 сек

Розміри підписів:
  ДСТУ 4145: 64 байт
  ML-DSA-44: 2420 байт
  Гібридний: 2484 байт



## 6. Верифікація підпису

In [7]:
print("="*70)
print("ВЕРИФІКАЦІЯ ПІДПИСУ")
print("="*70)

# Верифікуємо підпис з таймером
start_time = time.time()
results = scheme.verify(message, signature, keypair)
verify_time = time.time() - start_time

print(f"\nЗагальний час верифікації: {verify_time:.4f} сек")
print(f"\nРезультати верифікації:")
print(f"  ДСТУ 4145:   {'ВАЛІДНИЙ' if results['dstu_valid'] else 'НЕВАЛІДНИЙ'}")
print(f"  ML-DSA-44:   {'ВАЛІДНИЙ' if results['mldsa_valid'] else 'НЕВАЛІДНИЙ'}")
print(f"  Гібридний:   {'ВАЛІДНИЙ' if results['valid'] else 'НЕВАЛІДНИЙ'}")

print("\n" + "="*70)

ВЕРИФІКАЦІЯ ПІДПИСУ

 Верифікація гібридного підпису...
   1. ДСТУ 4145...
     Час перевірки підпису ДСТУ 4145: 458.5364 сек
       ВАЛІДНИЙ
   2. ML-DSA-44...
     Час перевірки підпису ML-DSA-44: 0.0003 сек
       ВАЛІДНИЙ

 Гібридний підпис: ВАЛІДНИЙ

Загальний час верифікації: 458.5433 сек

Результати верифікації:
  ДСТУ 4145:   ВАЛІДНИЙ
  ML-DSA-44:   ВАЛІДНИЙ
  Гібридний:   ВАЛІДНИЙ



## 7. Експорт у JSON

In [8]:
# Експорт підпису
json_signature = scheme.export_signature_json(signature)

print("Підпис у JSON форматі (перші 500 символів):")
print(json_signature[:500] + "...")

# Збереження
with open('signature_demo_257.json', 'w') as f:
    f.write(json_signature)

print(f"\nЗбережено: signature_demo_257.json")
print(f"Розмір JSON: {len(json_signature)} байт")

Підпис у JSON форматі (перші 500 символів):
{
  "algorithm": "DSTU-4145 + ML-DSA-44 + Купина-256",
  "timestamp": "2026-02-22T13:46:39.766709",
  "data_hash_kupyna": "9f446bbdf2aef7f7450b0986f0fdba83fdb1646f731de5f3a512fc57560d6057",
  "dstu4145": {
    "r": "0x186a037e34d5eb297b32b085934583f6a77bbabb32e0164859c5662cf24db341",
    "s": "0x186a57b9b02668a28cbe80af473b578ebd2c028e512b9a88522b8afa4d3e64b3"
  },
  "mldsa44": {
    "signature": "2kCIH14h/H2wivi2l1xNjh3ssJ19IkMjQCzJljQcoqIUU9O2fCeSS4ukXvozUz5VU255l9j26PgzZm6ZGUoJ6Cawler1mbCtWv5...

Збережено: signature_demo_257.json
Розмір JSON: 3636 байт


## 8. Тест виявлення підміни

In [9]:
print("="*70)
print("ТЕСТ ВИЯВЛЕННЯ ПІДМІНИ ДАНИХ")
print("="*70)

# Оригінальні дані
original = b"Payment amount: 1000 UAH"
print(f"\n1. Оригінал: {original.decode()}")

# Створюємо підпис з таймером
start_time = time.time()
sig = scheme.sign(original, keypair)
sign_time_test = time.time() - start_time
print(f"   Час підпису: {sign_time_test:.4f} сек")

# Верифікація оригіналу з таймером
start_time = time.time()
res1 = scheme.verify(original, sig, keypair)
verify_time_test1 = time.time() - start_time
print(f"   Час верифікації: {verify_time_test1:.4f} сек")
print(f"   Результат: {'ВАЛІДНИЙ' if res1['valid'] else 'НЕВАЛІДНИЙ'}")

# Підміна даних
tampered = b"Payment amount: 10000 UAH"
print(f"\n2. Підміна: {tampered.decode()}")
print("   (змінено 1000 -> 10000)")

# Верифікація підроблених даних з таймером
start_time = time.time()
res2 = scheme.verify(tampered, sig, keypair)
verify_time_test2 = time.time() - start_time
print(f"   Час верифікації: {verify_time_test2:.4f} сек")
print(f"   Результат: {'НЕВАЛІДНИЙ (очікувано)' if not res2['valid'] else 'ВАЛІДНИЙ (!!!)'}")

if not res2['valid']:
    print("\nПІДМІНУ ВИЯВЛЕНО!")
    print("Гібридна схема забезпечує цілісність даних.")

print("\n" + "="*70)

ТЕСТ ВИЯВЛЕННЯ ПІДМІНИ ДАНИХ

1. Оригінал: Payment amount: 1000 UAH

  Створення гібридного підпису...
   Дані: 24 байт
   1. ДСТУ 4145...
     Час підпису за допомогою ДСТУ 4145: 226.3741 сек
       Підпис: (r=0x22c4a7db2530e09968..., s=0x4d8b4d0f2f3034ded8...)
   2. ML-DSA-44...
     Час підпису за допомогою ML-DSA-44: 0.0004 сек
       Підпис: 2420 байт

 Гібридний підпис створено (2484 байт)
   Час підпису: 226.3974 сек

 Верифікація гібридного підпису...
   1. ДСТУ 4145...
     Час перевірки підпису ДСТУ 4145: 452.4354 сек
       ВАЛІДНИЙ
   2. ML-DSA-44...
     Час перевірки підпису ML-DSA-44: 0.0003 сек
       ВАЛІДНИЙ

 Гібридний підпис: ВАЛІДНИЙ
   Час верифікації: 452.4406 сек
   Результат: ВАЛІДНИЙ

2. Підміна: Payment amount: 10000 UAH
   (змінено 1000 -> 10000)

 Верифікація гібридного підпису...
   1. ДСТУ 4145...
     Час перевірки підпису ДСТУ 4145: 448.3525 сек
       НЕВАЛІДНИЙ
   2. ML-DSA-44...
     Час перевірки підпису ML-DSA-44: 0.0003 сек
       НЕВАЛІДНИЙ

 Гіб

## 9. Підсумкова таблиця продуктивності

In [10]:
print("="*70)
print("ПІДСУМКОВА ТАБЛИЦЯ ПРОДУКТИВНОСТІ")
print("="*70)

print(f"\nОперація                           Час (сек)")
print("-" * 70)
print(f"Генерація базової точки            {generation_time:>10.4f}")
print(f"Генерація гібридних ключів         {keygen_time:>10.4f}")
print(f"Створення гібридного підпису       {sign_time:>10.4f}")
print(f"Верифікація гібридного підпису     {verify_time:>10.4f}")
print(f"Тест підміни (підпис)              {sign_time_test:>10.4f}")
print(f"Тест підміни (верифікація 1)       {verify_time_test1:>10.4f}")
print(f"Тест підміни (верифікація 2)       {verify_time_test2:>10.4f}")

print(f"\n{'='*70}")
print(f"Загальний час всіх операцій:       {generation_time + keygen_time + sign_time + verify_time + sign_time_test + verify_time_test1 + verify_time_test2:>10.4f}")
print("="*70)

ПІДСУМКОВА ТАБЛИЦЯ ПРОДУКТИВНОСТІ

Операція                           Час (сек)
----------------------------------------------------------------------
Генерація базової точки              194.3111
Генерація гібридних ключів           220.6008
Створення гібридного підпису         230.6570
Верифікація гібридного підпису       458.5433
Тест підміни (підпис)                226.3974
Тест підміни (верифікація 1)         452.4406
Тест підміни (верифікація 2)         448.3586

Загальний час всіх операцій:        2231.3088


## 10. Характеристики безпеки

In [11]:
print("="*70)
print("ХАРАКТЕРИСТИКИ БЕЗПЕКИ")
print("="*70)

print("\nДСТУ 4145-2002:")
print("  - Еліптичні криві: GF(2^257)")
print("  - Рівняння: y^2 + xy = x^3 + Ax^2 + B")
print("  - Класична безпека: ~128 біт")
print("  - Хеш: Купина-256 (ДСТУ 7564:2014)")
print("  - Тип: Класична криптографія")
print("  - Стандарт: Український ДСТУ")

print("\nML-DSA-44 (Dilithium2):")
print("  - Базис: Module Lattice")
print("  - Рівень: NIST Level 2")
print("  - Класична безпека: ~128 біт")
print("  - Постквантова безпека: 128 біт")
print("  - Стійкість: Квантові атаки")
print("  - Стандарт: FIPS 204")

print("\nГібридна схема:")
print("  - Верифікація: ОБИДВА підписи мають бути валідні")
print("  - Захист: Класичний + Постквантовий")
print("  - Безпека: ~128 біт (класична) + 128 біт (постквантова)")
print("  - Перевага: Безпека навіть якщо один алгоритм зламано")

print("\n" + "="*70)

ХАРАКТЕРИСТИКИ БЕЗПЕКИ

ДСТУ 4145-2002:
  - Еліптичні криві: GF(2^257)
  - Рівняння: y^2 + xy = x^3 + Ax^2 + B
  - Класична безпека: ~128 біт
  - Хеш: Купина-256 (ДСТУ 7564:2014)
  - Тип: Класична криптографія
  - Стандарт: Український ДСТУ

ML-DSA-44 (Dilithium2):
  - Базис: Module Lattice
  - Рівень: NIST Level 2
  - Класична безпека: ~128 біт
  - Постквантова безпека: 128 біт
  - Стійкість: Квантові атаки
  - Стандарт: FIPS 204

Гібридна схема:
  - Верифікація: ОБИДВА підписи мають бути валідні
  - Захист: Класичний + Постквантовий
  - Безпека: ~128 біт (класична) + 128 біт (постквантова)
  - Перевага: Безпека навіть якщо один алгоритм зламано



## 11. Очищення

In [ ]:
import os

# Видаляємо тестовий файл
try:
    if os.path.exists('signature_demo_257.json'):
        os.remove('signature_demo_257.json')
        print("Тестові файли видалено")
except Exception as e:
    print(f"Помилка: {e}")

print("\n" + "="*70)
print("ДЕМОНСТРАЦІЯ ОСНОВНОГО ФУНКЦІОНАЛУ ЗАВЕРШЕНА УСПІШНО")
print("="*70)

## 12. Тестування на даних різної величини

In [13]:
print("="*70)
print("СТВОРЕННЯ ГІБРИДНОГО ПІДПИСУ ДЛЯ ТЕСТОВИХ ДАНИХ РІЗНОЇ ВЕЛИЧИНИ")
print("="*70)

# Мікро-повідомлення
msg_1B = secrets.token_bytes(1) # 1 Б

# Малий документ
doc_10KB = secrets.token_bytes(10 * 1024)    # 10 КБ

# Середній документ
doc_1MB = secrets.token_bytes(1024 * 1024)   # 1 МБ

# Великий документ
doc_10MB = secrets.token_bytes(10 * 1024 * 1024)  # 10 МБ

# Перевірка розмірів
#print(len(msg_1B))
#print(len(doc_10KB))
#print(len(doc_1MB))
#print(len(doc_10MB))

# Створюємо підпис мікро-повідомлення (з таймером)
start_time = time.time()
print(f"Розмір: {len(msg_1B)} байт")
signature = scheme.sign(msg_1B, keypair)
sign_time = time.time() - start_time

print(f"\nЗагальний час створення гібридного підпису: {sign_time:.4f} сек")
print(f"\nРозміри підписів:")
dstu_sig_size = len(scheme.dstu.export_signature(signature.dstu_signature))
print(f"  ДСТУ 4145: {dstu_sig_size} байт")
print(f"  ML-DSA-44: {len(signature.mldsa_signature)} байт")
print(f"  Гібридний: {dstu_sig_size + len(signature.mldsa_signature)} байт")

print("\n" + "="*70)

# Створюємо підпис документу розміром 10KB (з таймером)
start_time = time.time()
print(f"Розмір: {len(doc_10KB)} байт")
signature = scheme.sign(doc_10KB, keypair)
sign_time = time.time() - start_time

print(f"\nЗагальний час створення гібридного підпису: {sign_time:.4f} сек")
print(f"\nРозміри підписів:")
dstu_sig_size = len(scheme.dstu.export_signature(signature.dstu_signature))
print(f"  ДСТУ 4145: {dstu_sig_size} байт")
print(f"  ML-DSA-44: {len(signature.mldsa_signature)} байт")
print(f"  Гібридний: {dstu_sig_size + len(signature.mldsa_signature)} байт")

print("\n" + "="*70)

# Створюємо підпис документу розміром 1MB (з таймером)
start_time = time.time()
print(f"Розмір: {len(doc_1MB)} байт")
signature = scheme.sign(doc_1MB, keypair)
sign_time = time.time() - start_time

print(f"\nЗагальний час створення гібридного підпису: {sign_time:.4f} сек")
print(f"\nРозміри підписів:")
dstu_sig_size = len(scheme.dstu.export_signature(signature.dstu_signature))
print(f"  ДСТУ 4145: {dstu_sig_size} байт")
print(f"  ML-DSA-44: {len(signature.mldsa_signature)} байт")
print(f"  Гібридний: {dstu_sig_size + len(signature.mldsa_signature)} байт")

print("\n" + "="*70)

# Створюємо підпис документу розміром 10MB (з таймером)
start_time = time.time()
print(f"Розмір: {len(doc_10MB)} байт")
signature = scheme.sign(doc_10MB, keypair)
sign_time = time.time() - start_time

print(f"\nЗагальний час створення гібридного підпису: {sign_time:.4f} сек")
print(f"\nРозміри підписів:")
dstu_sig_size = len(scheme.dstu.export_signature(signature.dstu_signature))
print(f"  ДСТУ 4145: {dstu_sig_size} байт")
print(f"  ML-DSA-44: {len(signature.mldsa_signature)} байт")
print(f"  Гібридний: {dstu_sig_size + len(signature.mldsa_signature)} байт")

print("\n" + "="*70)

СТВОРЕННЯ ГІБРИДНОГО ПІДПИСУ ДЛЯ ТЕСТОВИХ ДАНИХ РІЗНОЇ ВЕЛИЧИНИ
Розмір: 1 байт

  Створення гібридного підпису...
   Дані: 1 байт
   1. ДСТУ 4145...
     Час підпису за допомогою ДСТУ 4145: 234.2323 сек
       Підпис: (r=0x1f4d99b0d11458e0a9..., s=0xafa15280bd6fb0838e...)
   2. ML-DSA-44...
     Час підпису за допомогою ML-DSA-44: 0.0003 сек
       Підпис: 2420 байт

 Гібридний підпис створено (2484 байт)

Загальний час створення гібридного підпису: 234.2544 сек

Розміри підписів:
  ДСТУ 4145: 64 байт
  ML-DSA-44: 2420 байт
  Гібридний: 2484 байт

Розмір: 10240 байт

  Створення гібридного підпису...
   Дані: 10240 байт
   1. ДСТУ 4145...
     Час підпису за допомогою ДСТУ 4145: 221.2900 сек
       Підпис: (r=0x12a0516cc8f84bcdce..., s=0x2766ddfc8da84e1b1e...)
   2. ML-DSA-44...
     Час підпису за допомогою ML-DSA-44: 0.0004 сек
       Підпис: 2420 байт

 Гібридний підпис створено (2484 байт)

Загальний час створення гібридного підпису: 223.2799 сек

Розміри підписів:
  ДСТУ 4145: 64 